<a href="https://colab.research.google.com/github/lnsayer/personal_repo/blob/main/amazon_node_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amazon Dataset (from PyTorch Geometric) Node Classification Project

The Amazon Computers and Amazon Photo networks from the “Pitfalls of Graph Neural Network Evaluation” paper. Nodes represent goods and edges represent that two goods are frequently bought together. Given product reviews as bag-of-words node features, the task is to map goods to their respective product category.

In [1]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 3.7 MB/s eta 0:00:00


In [59]:
from torch_geometric.datasets import Amazon
import torch

dataset = Amazon(root="data/Amazon", name="Computers")

data = dataset[0]

print(f"Number of nodes in graph is: {data.x.shape[0]}")
print(f"Number of features for each node is {data.x.shape[1]}")
print(f"Number of edges is {data.edge_index.shape[1]}")
print(f"Number of classes is {dataset.num_features}\n")
print(f"We want to predict the classes of the 13752 Computers\n")

print(f"The number of class members for each class:\n{data.y.unique(return_counts=True)}")

Number of nodes in graph is: 13752
Number of features for each node is 767
Number of edges is 491722
Number of classes is 767

We want to predict the classes of the 13752 Computers

The number of class members for each class:
(tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), tensor([ 436, 2142, 1414,  542, 5158,  308,  487,  818, 2156,  291]))


In [72]:
# Create a train mask:
indices = torch.randperm(data.x.shape[0])
train_mask = indices

indices = torch.randperm(data.x.shape[0])
print(indices)

tensor([10684,   545, 10397,  ...,  4027,  5944, 11758])
tensor([ 2085,  4630,  5585,  ..., 11992,  7169,  4397])
